In [77]:
import random
import numpy as np
import copy

In [78]:

inputDataset = [[1,2],[3,4]]
outputDataset = [[2,4],[6,8]]
learningRate=0.002
errorThresh=0.1

class Layer:
    
    def __init__(self,neuronCount=None,position=None,neuronvals=None):
        
        self.neuronCount = neuronCount
        self.position = position
        self.neuronvals = []
        self.weights = []
        self.bias = []
        self.NodeDeltas=[] 
        self.batchNeuronVals=[]
        #set by setintialweights and setnodedelta
        self.previousLayer =None
        self.AfterLayer =None

        for i in range(self.neuronCount):

            self.bias.append(random.uniform(0.0001,1))
            # self.bias.append(1)

        if position==1:
            self.neuronvals = neuronvals
    
    def setInitialWeights(self,previousLayer):

        self.previousLayer = previousLayer
        test =[
            [1,2,3,4]
        ]
        for i in range(self.neuronCount):
            temp = []
            for j in range(previousLayer.neuronCount):
                temp.append(random.uniform(0.0001,1))
                # temp.append(test[0][i+j])

            self.weights.append(temp)

    def setNodeDelta(self,AfterLayer,CurrentDataPoint=-1):
        """""
        set nodedelta value for layers
        """""
        self.AfterLayer = AfterLayer
        if self.position == -1:
            # outputlayer
            if len(self.NodeDeltas)==0:
                for i in range(self.neuronCount):

                    nodedelta = self.neuronvals[i]-outputDataset[CurrentDataPoint][i] #exepectd to change dimenston CHNGDIMN
                    self.NodeDeltas.append(nodedelta)
            else:
                for i in range(self.neuronCount):

                    nodedelta = self.neuronvals[i]-outputDataset[CurrentDataPoint][i] #exepectd to change dimenston CHNGDIMN
                    self.NodeDeltas[i]=nodedelta
        else:
            # hidden layers
            if len(self.NodeDeltas)==0:

                for i in range(self.neuronCount):
                    nodedelta= 0
                    for j in range(AfterLayer.neuronCount):
                        nodedelta = nodedelta+AfterLayer.NodeDeltas[j]*self.weights[i][j]
                    self.NodeDeltas.append(nodedelta)

            else:
                    for i in range(self.neuronCount):
                        nodedelta= 0
                        for j in range(AfterLayer.neuronCount):
                            nodedelta = nodedelta+AfterLayer.NodeDeltas[j]*self.weights[i][j]
                        self.NodeDeltas[i]=nodedelta
    
    def updateWeightsandBias(self):

        len_weight = len(self.weights[0])# based on previous layer of no(neurons) lenof weights is set

        for i in range(self.neuronCount):

            for j in range(len_weight):
                
                new_Weight = self.weights[i][j] - learningRate * self.NodeDeltas[i] * self.previousLayer.neuronvals[j]
                self.weights[i][j] = new_Weight
            
            new_Bias = self.bias[i] - learningRate * self.NodeDeltas[i]
            self.bias[i]=new_Bias        

In [79]:
class Ann:

    def __init__(self):
        self.LayerArr = []
        

    def bind(self,layer1=None,layer2=None):
        layer2.setInitialWeights(layer1)

    def forward(self,layer1=None,layer2=None): 
        
        if len(layer2.neuronvals)==0:

            for i in range(layer2.neuronCount):
                layer2.neuronvals.append(np.dot(layer1.neuronvals,layer2.weights[i])+layer2.bias[i])
            
            
            if(layer2.position==-1):
                temp =copy.deepcopy(layer2.neuronvals)
                layer2.batchNeuronVals.append(temp)

        else:
            for i in range(layer2.neuronCount):
                layer2.neuronvals[i]=np.dot(layer1.neuronvals,layer2.weights[i])+layer2.bias[i]

            if(layer2.position==-1):
                temp =copy.deepcopy(layer2.neuronvals)
                layer2.batchNeuronVals.append(temp)
            
    def backPropagate(self,currentLayer=None,AfterLayer=None,CurrentDataPoint=-1):

        currentLayer.setNodeDelta(AfterLayer,CurrentDataPoint=CurrentDataPoint)
        currentLayer.updateWeightsandBias()
        
    def compile(self):
        
        for i in range(len(self.LayerArr)-1):

            self.bind(layer1=self.LayerArr[i],layer2=self.LayerArr[i+1])
    
    def BatchError(self):
        error = 0
        for i in range(len(self.LayerArr[-1].batchNeuronVals)):     
            for j in range(len(inputDataset[i])):
                error=error+0.5*(self.LayerArr[-1].batchNeuronVals[i][j]-outputDataset[i][j])**2
        return error
      
    def Train(self):

        errorrate = 9999999 
        layercount = len(self.LayerArr)

        while (errorrate > errorThresh):
           
            
            
            #forward propagation
            for N in range(len(inputDataset)):
                
                self.LayerArr[0].neuronvals = inputDataset[N]      

                for i in range(layercount-1):
                    self.forward(layer1=self.LayerArr[i],layer2=self.LayerArr[i+1])
                
                #backward propagation
                currentLayeridx = layercount-1  
                for i in range(layercount-1):

                    if self.LayerArr[currentLayeridx].position ==-1:# handling output layer 
                        self.backPropagate(currentLayer=self.LayerArr[currentLayeridx],CurrentDataPoint=N)
                        currentLayeridx=currentLayeridx-1
                    else:
                        self.backPropagate(currentLayer=self.LayerArr[currentLayeridx],AfterLayer=self.LayerArr[currentLayeridx+1])
                        currentLayeridx=currentLayeridx-1

                
                
            errorrate = self.BatchError()
            print(errorrate)
            
            if(errorrate < errorThresh):
                    print("optimised")
                    print(self.LayerArr[-1].batchNeuronVals)
                    break
            self.LayerArr[-1].batchNeuronVals = []
                                         
    def add(self,layer=None):
        self.LayerArr.append(layer)

In [80]:

inputl = Layer(neuronCount=2,position=1,neuronvals=[2,3])
hidden = Layer(neuronCount=2,position=2,neuronvals=[])
# hidden2 = Layer(neuronCount=2,position=3,neuronvals=[]) 
output = Layer(neuronCount=2,position=-1,neuronvals=[])

Network = Ann()
Network.add(layer=inputl)
Network.add(layer=hidden)
# Network.add(layer=hidden2)
Network.add(layer=output)
Network.compile()
Network.Train()

9.771678671236524
7.737172355764708
6.041329543042629
4.668444231417068
3.5868164533780083
2.755327302852791
2.129684777715756
1.6672989299310168
1.3303823150155625
1.087394864541566
0.9132412393866194
0.7886974396292836
0.6994732286104939
0.635193059565219
0.5884580722963464
0.5540611627494074
0.5283701002068876
0.508863718525103
0.4937943059310998
0.48194768738999577
0.47247573750149874
0.46478095925542817
0.45843765211851895
0.4531383686463931
0.448657638822344
0.4448273857396412
0.4415202140746324
0.43863798527560605
0.4361039417426312
0.4338572183541432
0.4318489670229664
0.4300395785374241
0.4283966577336684
0.4268935218556428
0.4255080672788233
0.4242218996533678
0.4230196556331443
0.4218884664283606
0.42081752822150026
0.41979775449363416
0.4188214921431996
0.41788228800536653
0.4169746956948367
0.4160941150573119
0.4152366582277879
0.41439903755837854
0.4135784716287816
0.4127726062793466
0.4119794481715701
0.4111973088263411
0.41042475744603235
0.40966058111386017
0.408903751